This notebook is focused on the modelling.

## Preliminaries

### Paths

In [1]:
import os
import pathlib
import sys

In [2]:
if not 'google.colab' in str(get_ipython()):
    
    parts = pathlib.Path(os.getcwd()).parts    
    limit = max([index for index, value in enumerate(parts) if value == 'infections'])    
    parent = os.path.join(*list(parts[:(limit + 1)]))
    
    sys.path.append(os.path.join(parent, 'src'))


In [3]:
parent

'J:\\library\\premodelling\\projects\\infections'

<br>
<br>

### Libraries

In [4]:
%matplotlib inline

import datetime

import logging
import collections

import numpy as np
import pandas as pd


<br>

### Custom

In [5]:
import src.modelling.DataStreams
import src.modelling.DataReconstructions
import src.modelling.Differences
import src.modelling.DataNormalisation
import src.modelling.Estimates

<br>
<br>

### Logging

In [6]:
logging.basicConfig(level=logging.INFO,
                    format='\n\n%(message)s\n%(asctime)s.%(msecs)03d',
                        datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)

<br>
<br>

## Part II

### Setting-Up

A class for the data splitting fractions

In [7]:
Fraction = collections.namedtuple(
    typename='Fraction',
    field_names=['training', 'validating', 'testing'])

<br>

**Modelling Arguments**

> * Predict `output_steps` days into the future, based on `input_width` days of history

Herein

* $input\_width \in widths$  $\qquad$  [$widths$ is a range of input window values (days)]
* $output\_steps = 15$ days
  
And

* $label\_width = output\_steps$


In [8]:
Arguments = collections.namedtuple(
    typename='Arguments',
    field_names=['input_width', 'label_width', 'shift', 'training_', 'validating_', 'testing_', 'label_columns'])

In [9]:
widths = range(18, 20)
output_steps = 15

<br>
<br>

### Training, Validating, Testing Data

Foremost: The data sets for training, validating, and testing

In [10]:
training, validating, testing = src.modelling.DataStreams.DataStreams(root=parent, fraction=Fraction._make(
        (0.75, 0.15, 0.10))).exc()

In [11]:
logger.info(training.columns)



Index(['group', 'date', 'covidOccupiedBeds', 'covidOccupiedMVBeds',
       'estimatedNewAdmissions', 'EDC0-4', 'EDC5-9', 'EDC10-14', 'EDC15-19',
       'EDC20-24', 'EDC25-29', 'EDC30-34', 'EDC35-39', 'EDC40-44', 'EDC45-49',
       'EDC50-54', 'EDC55-59', 'EDC60-64', 'EDC65-69', 'EDC70-74', 'EDC75-79',
       'EDC80-84', 'EDC85-89', 'EDC90+', 'newDeaths28DaysByDeathDate',
       'EDV12-15', 'EDV16-17', 'EDV18-24', 'EDV25-29', 'EDV30-34', 'EDV35-39',
       'EDV40-44', 'EDV45-49', 'EDV50-54', 'EDV55-59', 'EDV60-64', 'EDV65-69',
       'EDV70-74', 'EDV75-79', 'EDV80-84', 'EDV85-89', 'EDV90+'],
      dtype='object')
2022-03-16 17:46:26.647


<br>
<br>

### Reconstruction

Reconstructions: Each data set is a concatenation of records from various NHS Trusts, however because the aim is a single predicting/forecasting model for all trusts, the data should be reconstructed ...

In [12]:
reconstructions = src.modelling.DataReconstructions.DataReconstructions()
training = reconstructions.exc(blob=training)
validating = reconstructions.exc(blob=validating)
testing = reconstructions.exc(blob=testing)

<br>
<br>

### Differences

Using difference values rather than actual values

In [13]:
differences = src.modelling.Differences.Differences()
training = differences.exc(blob=training)
validating = differences.exc(blob=validating)
testing = differences.exc(blob=testing)

<br>
<br>

### Normalisation

In [14]:
normalisation = src.modelling.DataNormalisation.DataNormalisation(reference=training)
training_ = normalisation.normalise(blob=training)
validating_ = normalisation.normalise(blob=validating)
testing_ = normalisation.normalise(blob=testing)

training_.drop(columns='point', inplace=True)
validating_.drop(columns='point', inplace=True)
testing_.drop(columns='point', inplace=True)

<br>
<br>

### Modelling

In [15]:
arguments = Arguments(input_width=None, label_width=output_steps, shift=output_steps,
                      training_=training_, validating_=validating_, testing_=testing_,
                      label_columns=['estimatedNewAdmissions'])

validations, tests = src.modelling.Estimates.Estimates(
    n_features=training_.shape[1], 
    output_steps=output_steps).exc(widths=widths, arguments=arguments)

logger.info(validations)
logger.info(tests)

Epoch 1/100
2423/2423 [==============================] - 13s 5ms/step - loss: 8.3478 - mean_absolute_error: 0.9584 - val_loss: 0.5601 - val_mean_absolute_error: 0.5371
Epoch 2/100
2423/2423 [==============================] - 12s 5ms/step - loss: 1.0853 - mean_absolute_error: 0.5550 - val_loss: 0.5710 - val_mean_absolute_error: 0.5377
Epoch 3/100
2423/2423 [==============================] - 12s 5ms/step - loss: 1.1728 - mean_absolute_error: 0.5749 - val_loss: 0.5403 - val_mean_absolute_error: 0.5230
Epoch 4/100
2423/2423 [==============================] - 13s 6ms/step - loss: 1.0901 - mean_absolute_error: 0.5565 - val_loss: 0.5377 - val_mean_absolute_error: 0.5212
Epoch 5/100
2423/2423 [==============================] - 12s 5ms/step - loss: 1.1509 - mean_absolute_error: 0.5571 - val_loss: 0.5351 - val_mean_absolute_error: 0.5194
Epoch 6/100
2423/2423 [==============================] - 12s 5ms/step - loss: 1.0568 - mean_absolute_error: 0.5419 - val_loss: 0.5272 - val_mean_absolute_error:



  method history ahead      loss       mae
0    CNN      18    15   0.55426  0.531915
1   LSTM      18    15  0.531987  0.521566
2    CNN      19    15  0.608863   0.55482
3   LSTM      19    15  0.537783  0.525687
2022-03-16 18:06:16.238


  method history ahead      loss       mae
0    CNN      18    15  0.791428  0.370899
1   LSTM      18    15  0.678758  0.331206
2    CNN      19    15   1.27334  0.467748
3   LSTM      19    15  0.689945   0.33892
2022-03-16 18:06:16.250


<br>
<br>

### Delete DAG Diagrams

In [16]:
%%bash

rm -rf *.pdf